# <font color='blue'>Question 1 - manual construction of CFG</font>

---

### Introduction
Syntactic parsing is an NLP task that sets it's goal to parse a language sentence to its syntactic parts; <br>
A common method to generate a sentence segmentation to it's syntactic parts is via construction of context free grammer that cpature the language rules; We note that a natural language is not context free, therefore a context free grammer (that is not very large) cannot capture all the behaviours, but we may construct a gradually increasing set of CFG grammer rules that will incresingly capture more and more natural gramatical phenomenon;<br>
In the following section we will construct a simple grammer that will be upgraded to capture more of **English** rules;

---

### <font color='blue'>Question 1.1: Extend a CFG to support Number agreement, Pronouns and Dative Constructions</font>
We are given a basic CFG rules, and few exmpale sentences; our task here is to add CFG rules to the given ones so that the new set of rules will be able to parse all the given sentences;

#### 1.1.1 Extend the CFG so that the following sentences can be parsed:

* John left
* John loves Mary
* They love Mary
* They love her
* She loves them
* Everybody loves John
* A boy loves Mary
* John gave Mary a heavy book
* John gave it to Mary

The given CFG:

In [2]:
sg = """
S -> NP VP
VP -> IV | TV NP
NP -> 'John' | "bread"
IV -> 'left'
TV -> 'eats'
"""

The new constructed CFG:

In [3]:
import nltk

sg = """
S -> SNP SVP | PLNP PLVP 
SVP -> SIV | STV ACCNP | STV SNP | STV PLNP  | SDV SNP SNP | SDV ACCNP "to" SNP
PLVP -> PLIV | PLTV ACCNP |  PLTV SNP | PLTV PLNP
PLNP -> 'They' 
SNP -> ADJ SNP |DET SNP | 'John'  | 'She' | 'Everybody' | "boy" | "Mary" | "bread" | "book"
SIV -> 'left'
STV -> 'eats' | 'loves' 
SDV -> "gave"
PLTV -> 'love'
ACCNP -> 'them' | 'her' | "it"
DET -> "A" | "a"
ADJ -> "heavy"
"""
g = nltk.CFG.fromstring(sg)

# Bottom-up  parser
sr_parser = nltk.ShiftReduceParser(g, trace=0)

def parse_sentence(sent):
    tokens = sent.split()
    trees = sr_parser.parse(tokens)
    print("parsing sentence: '%s'" % sent)
    for tree in trees:
        print(tree)
    print("\n")

parse_sentence("John left")
parse_sentence("John eats bread")
parse_sentence("John loves Mary")
parse_sentence("They love Mary")
parse_sentence("They love her")
parse_sentence("She loves them")
parse_sentence("Everybody loves John")
parse_sentence("A boy loves Mary")
parse_sentence("John gave Mary a heavy book")
parse_sentence("John gave it to Mary")

parsing sentence: 'John left'
(S (SNP John) (SVP (SIV left)))


parsing sentence: 'John eats bread'
(S (SNP John) (SVP (STV eats) (SNP bread)))


parsing sentence: 'John loves Mary'
(S (SNP John) (SVP (STV loves) (SNP Mary)))


parsing sentence: 'They love Mary'
(S (PLNP They) (PLVP (PLTV love) (SNP Mary)))


parsing sentence: 'They love her'
(S (PLNP They) (PLVP (PLTV love) (ACCNP her)))


parsing sentence: 'She loves them'
(S (SNP She) (SVP (STV loves) (ACCNP them)))


parsing sentence: 'Everybody loves John'
(S (SNP Everybody) (SVP (STV loves) (SNP John)))


parsing sentence: 'A boy loves Mary'
(S (SNP (DET A) (SNP boy)) (SVP (STV loves) (SNP Mary)))


parsing sentence: 'John gave Mary a heavy book'
(S
  (SNP John)
  (SVP
    (SDV gave)
    (SNP Mary)
    (SNP (DET a) (SNP (ADJ heavy) (SNP book)))))


parsing sentence: 'John gave it to Mary'
(S (SNP John) (SVP (SDV gave) (ACCNP it) to (SNP Mary)))




---

Q. Do we need to encode gender in the grammar?<br>
A. **No**, as in english there is no distinction between genders nither in verbs nor nouns;

#### 1.1.2 Example of overgeneration
We present here an exmple in wich our constructed grammer is successfuly parsing a "bad sentece" i.e. a setence that does not follows Emglish grammer; <br>
In our example we see 2 errors:
* sequence of 2 determiners
* determinter before a pronoun

In [4]:
parse_sentence("A A Everybody loves John")

parsing sentence: 'A A Everybody loves John'
(S
  (SNP (DET A) (SNP (DET A) (SNP Everybody)))
  (SVP (STV loves) (SNP John)))




---

### <font color='blue'>Question 1.2: Extend a CFG to support Coordination and Prepositional Phrases</font>

#### 1.2.1 Extend the CFG so that the following sentences can be parsed:
* John saw a man with a telescope
* John saw a man on the hill with a telescope
* Mary knows men and women
* Mary knows men, children and women
* John and Mary eat bread
* John and Mary eat bread with cheese

some explenations to the LHS rules naming:<br>

S = sentence<br>

NP = singular noun phrase<br>
VP = singular verb phrase<br>
  
IV = Intransitive verb<br>
1TV = transitive with 1 object <br>
2TV = transitive with 2 objects <br>

S_/ P_ = Singular/ Plural<br>
A_ = atom prefix, transforms only to atoms (terminals)<br>

ACC_NP = pronoun<br>
DET = determiner<br>
ADJ = adjective<br>
LOC = location<br>

---


**First attemp:** adding CFG rules:<br>
first 4 sentences can be parsed via 'shifr-reduce' nltk parser;
That parser has the following strategy:
1) shift to the next word, only when no rules of the stack status can be reduced;
2) reduce: if there is a rule that can be deduced backwards, do it on the "firsr-rule-match" approach

That parsing strategy, can lead to sentences that are not parsed enev though they can be generated by the CFG given;
As mentioned, the 4 below sentences does not lead to such conflict;<br>

Presenting the CFG parsing the 4 first sentences:

In [28]:
sg = """
S -> S_NP S_VP | P_NP P_VP

S_VP -> S_IV | S_1TV ACC_NP | S_1TV P_NP | S_1TV S_NP  | S_2TV_TO S_NP S_NP | S_2TV_TO ACC_NP "to" S_NP | S_2TV_WITH S_NP "with" S_NP
S_NP -> ADJ S_NP | DET S_NP | S_NP "on" DET LOC |  S_NP S_NP | A_S_NP 
A_S_NP -> 'John' | 'She' | 'Everybody' | "boy" | "Mary" | 'bread' | "book" | "man" | "telescope" | "cheese"
S_IV -> 'left'
S_1TV -> 'eats' | 'loves' | "knows"
S_2TV_TO -> "gave"
S_2TV_WITH -> "saw"

P_NP -> A_P_NP "and" A_P_NP | A_P_NP P_NP | S_NP "and" S_NP | S_NP "with" S_NP
A_P_NP -> 'They' | "men" | "women" | "children" | "men,"
P_VP -> P_TV ACC_NP | P_TV S_NP | P_TV P_NP 
P_TV -> 'love' | "eat"

ACC_NP -> 'them' | 'her' | "it"
DET -> "A" | "a"| "the"
ADJ -> "heavy"
LOC -> "hill"
"""

def parse_sentence(sent):
    tokens = sent.split()
    trees = sr_parser.parse(tokens)
    print("parsing sentence: '%s'" % sent)
    for tree in trees:
        print(tree)
    print("\n")
    
g = nltk.CFG.fromstring(sg)

# Bottom-up  parser
sr_parser = nltk.ShiftReduceParser(g, trace=0)

parse_sentence("John saw a man with a telescope")
parse_sentence("John saw a man on the hill with a telescope")

parse_sentence("Mary knows men and women")
parse_sentence("Mary knows men, children and women")

parsing sentence: 'John saw a man with a telescope'
(S
  (S_NP (A_S_NP John))
  (S_VP
    (S_2TV_WITH saw)
    (S_NP (DET a) (S_NP (A_S_NP man)))
    with
    (S_NP (DET a) (S_NP (A_S_NP telescope)))))


parsing sentence: 'John saw a man on the hill with a telescope'
(S
  (S_NP (A_S_NP John))
  (S_VP
    (S_2TV_WITH saw)
    (S_NP (S_NP (DET a) (S_NP (A_S_NP man))) on (DET the) (LOC hill))
    with
    (S_NP (DET a) (S_NP (A_S_NP telescope)))))


parsing sentence: 'Mary knows men and women'
(S
  (S_NP (A_S_NP Mary))
  (S_VP (S_1TV knows) (P_NP (A_P_NP men) and (A_P_NP women))))


parsing sentence: 'Mary knows men, children and women'
(S
  (S_NP (A_S_NP Mary))
  (S_VP
    (S_1TV knows)
    (P_NP (A_P_NP men,) (P_NP (A_P_NP children) and (A_P_NP women)))))




**Second attempt:**<br>
Now we look on the following sentences:

"John and Mary eat bread"<br>
"John and Mary eat bread with cheese"<br>

given that the parser is a shift-reduce one, and using the above mentioned strategy,<be>
One cannot generate a CGF to distinguish the end of sentence via shift-reduce parser, as the parser is greedy in the sense that it will complete a sentence once it can; <br>
Therefore if we would like to enable parsing of the first sentence the second will be left with redundant part, and if we will enforce a rule to parse the second, the first will not be parsed;
Resulting in a both cases, in one that cannot be parsed;<br>

We present the following trick:<br>
* We add manually to the end of a parsed sentence a marker to mark the end of the sentence: 'ENDSENT';<br>
* Then, we change the CFG to wait for that marker before allowing the parser to reduce;<br>


[Note] - That trick is fitted to the task of parsing a sentence with the given parser, one can think of other parser that alone would deal with that problem, but may be less efficient by run time;<br>

Below CFG parses

In [25]:
sg = """
S -> S_NP S_VP_ES | P_NP P_VP_ES

S_VP -> S_IV | S_1TV ACC_NP | S_1TV P_NP | S_1TV S_NP  | S_2TV_TO S_NP S_NP | S_2TV_TO ACC_NP "to" S_NP | S_2TV_WITH S_NP "with" S_NP
S_VP_ES -> S_VP ES
S_NP -> ADJ S_NP | DET S_NP | S_NP "on" DET LOC |  S_NP A_S_NP | A_S_NP 
S_NP_ES -> S_NP ES
A_S_NP -> 'John' | 'She' | 'Everybody' | "boy" | "Mary" | 'bread' | "book" | "man" | "telescope" | "cheese"

S_IV -> 'left'
S_1TV -> 'eats' | 'loves' | "knows"
S_2TV_TO -> "gave"
S_2TV_WITH -> "saw"

P_NP -> A_P_NP "and" A_P_NP | A_P_NP P_NP | S_NP "and" S_NP | S_NP "with" S_NP
P_NP_ES -> P_NP ES
A_P_NP -> 'They' | "men" | "women" | "children" | "men,"
P_VP_ES -> P_TV ACC_NP ES | P_TV_TOP_ES | P_TV_TOS_ES
P_TV_TOP_ES -> P_TV P_NP_ES
P_TV_TOS_ES -> P_TV S_NP_ES
P_TV -> 'love' | "eat"

ACC_NP -> 'them' | 'her' | "it"
DET -> "A" | "a"| "the"
ADJ -> "heavy"
LOC -> "hill"

ES -> 'ENDSENT'
"""

def parse_sentence(sent):
    tokens = sent.split()
    tokens.append('ENDSENT')
    trees = sr_parser.parse(tokens)
    print("parsing sentence: '%s'" % sent)
    for tree in trees:
        print(tree)
    print("\n")
    
g = nltk.CFG.fromstring(sg)

# Bottom-up  parser
sr_parser = nltk.ShiftReduceParser(g, trace=0)

parse_sentence("John saw a man with a telescope")
parse_sentence("John saw a man on the hill with a telescope")

parse_sentence("Mary knows men and women")
parse_sentence("Mary knows men, children and women")

parse_sentence("John and Mary eat bread")
parse_sentence("John and Mary eat bread with cheese")


parsing sentence: 'John saw a man with a telescope'
(S
  (S_NP (A_S_NP John))
  (S_VP_ES
    (S_VP
      (S_2TV_WITH saw)
      (S_NP (DET a) (S_NP (A_S_NP man)))
      with
      (S_NP (DET a) (S_NP (A_S_NP telescope))))
    (ES ENDSENT)))


parsing sentence: 'John saw a man on the hill with a telescope'
(S
  (S_NP (A_S_NP John))
  (S_VP_ES
    (S_VP
      (S_2TV_WITH saw)
      (S_NP
        (S_NP (DET a) (S_NP (A_S_NP man)))
        on
        (DET the)
        (LOC hill))
      with
      (S_NP (DET a) (S_NP (A_S_NP telescope))))
    (ES ENDSENT)))


parsing sentence: 'Mary knows men and women'
(S
  (S_NP (A_S_NP Mary))
  (S_VP_ES
    (S_VP (S_1TV knows) (P_NP (A_P_NP men) and (A_P_NP women)))
    (ES ENDSENT)))


parsing sentence: 'Mary knows men, children and women'
(S
  (S_NP (A_S_NP Mary))
  (S_VP_ES
    (S_VP
      (S_1TV knows)
      (P_NP
        (A_P_NP men,)
        (P_NP (A_P_NP children) and (A_P_NP women))))
    (ES ENDSENT)))


parsing sentence: 'John and Mary eat brea